In [1]:
import re

import requests
import pdfplumber
import pandas as pd
from collections import namedtuple

In [20]:
#url = "https://www.rad.cvm.gov.br/ENET/frmDownloadDocumento.aspx?Tela=ext&descTipo=IPE&CodigoInstituicao=1&numProtocolo=966049&numSequencia=490783&numVersao=1"
#url = "https://www.rad.cvm.gov.br/ENET/frmDownloadDocumento.aspx?Tela=ext&descTipo=IPE&CodigoInstituicao=1&numProtocolo=1005011&numSequencia=529745&numVersao=1"

url = "https://www.rad.cvm.gov.br/ENET/frmDownloadDocumento.aspx?Tela=ext&descTipo=IPE&CodigoInstituicao=1&numProtocolo=924876&numSequencia=449610&numVersao=1"

In [21]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [22]:

from datetime import datetime


def extract_operations(text, start_phrase, end_phrase):
    pattern = re.compile(f"{re.escape(start_phrase)}(?:.*?\n)+?.*?{re.escape(end_phrase)}", re.DOTALL)
    matches = pattern.findall(text)
    return matches

def generate_operations_df(text, month, year, responsible):    
    intermediario_options = r"(?:Direto c/ a Cia|JP Morgan|Corretora Itaú|Goldman Sachs|BTG Pactual)"
    operacao_options = r"(?:Venda à vista|Compra à vista)"
    pattern = re.compile(r"((?:Ações|Outros))\s+((?:ON|ADR\sORDINARIA))\s+({})\s+({})\s+(\d+)\s+([\d\.,]+)\s+([\d\.,]+)\s+([\d\.,]+)".format(intermediario_options, operacao_options), re.MULTILINE)
    exercicio_options_pattern = re.compile(r"Exercício de\n(Ações)\s+(ON)\s+(.+?)\s+(\d+)\s+([\d\.,]+)\s+([\d\.,]+)\s+([\d\.,]+)")
    exercicio_options_subscription_pattern = re.compile(r"(Ações|Outros)\s+((?:ON|ADR\sORDINARIA))\s+(.+?)\s+((?:Subscrição\s+)?Exercício)(?:\s+Opção)?\s+(\d+)\s+([\d\.,]+)\s+([\d\.,]+)\s+([\d\.,]+)")
    entrega_pattern = re.compile(r"(Ações|Outros)\s+(ON)\s+(.+?)\s+(.+?)\s+(\d+)\s+([\d\.,]+)\s+([\d\.,]+)\s+([\d\.,]+)\n\((PSU)\)")


    matches = pattern.findall(text)
    exercicio_matches = exercicio_options_pattern.findall(text)
    exercicio_options_subscription_pattern_matches = exercicio_options_subscription_pattern.findall(text)
    entrega_pattern_matches = entrega_pattern.findall(text) 

    operations_list = []
    header = ["Valor Mobiliário/Derivativo", "Características dos Títulos", "Intermediário", "Operação", "Dia", "Quantidade", "Preço", "Volume (R$)"]
    operations_list.append(header)

    for match in matches + exercicio_matches + exercicio_options_subscription_pattern_matches + entrega_pattern_matches:
        operation = [match[0], match[1], match[2].strip(), match[3].strip() if len(match) == 8 else "Exercício de Opções", match[4 if len(match) == 8 else 3], match[5 if len(match) == 8 else 4], match[6 if len(match) == 8 else 5], match[7 if len(match) == 8 else 6]]
        operations_list.append(operation)

    operations_df = pd.DataFrame(operations_list[1:], columns=operations_list[0])

    operations_df['Mês'] = month
    operations_df['Ano'] = year
    operations_df['Responsável'] = responsible


    return operations_df

def generate_operations_df_2(text):
    pattern_opcao = re.compile(r"Exercício de\n(.*?)\nOpções", re.DOTALL)
    pattern_others = re.compile(r"(Outros.*?)\n", re.DOTALL)

    # Find and extract the 'Exercício de Opções' entry
    opcao_match = pattern_opcao.search(text)
    opcao_line = opcao_match.group(1) + ' Exercício de Opções'

    # Find and extract the other entries
    others_matches = pattern_others.findall(text)

    # Combine the entries and split them into lines
    entries = [opcao_line] + others_matches
    lines = [entry.split() for entry in entries]

    # Create a pandas DataFrame
    header = ["Valor Mobiliário/Derivativo", "Características dos Títulos", "Intermediário", "Operação", "Dia", "Quantidade", "Preço", "Volume (R$)"]
    df = pd.DataFrame(lines, columns=header)
    
    return df

def generate_operations_df_3(text):
    pattern = re.compile(r"((?:Ações|Outros))\s+((?:ON|ADR\sORDINARIA))\s+(.+?)\s+(.+?)\s+(\d+)\s+([\d\.,]+)\s+([\d\.,]+)\s+([\d\.,]+)", re.MULTILINE)
    matches = pattern.findall(text)

    operations_list = []
    header = ["Valor Mobiliário/Derivativo", "Características dos Títulos", "Intermediário", "Operação", "Dia", "Quantidade", "Preço", "Volume (R$)"]
    operations_list.append(header)

    for match in matches:
        operation = [match[0], match[1], match[2].strip(), match[3].strip(), match[4], match[5], match[6], match[7]]
        operations_list.append(operation)

    operations_df = pd.DataFrame(operations_list[1:], columns=operations_list[0])

    return operations_df

def extract_datetime(text):
    pattern = r"Em\s+(\d{2})\/(\d{4})"
    match = re.search(pattern, text)
    if match:
        month, year = match.groups()
        return datetime(int(year), int(month), 1)
    return None

def generate_trade_df(operations_list, date):
    operations_responsibles = ["Conselho Administração", "Diretoria", "Conselho Fiscal"]
    dfs = []
    for operation, responsible in zip(operations_list, operations_responsibles):
        dfs.append(generate_operations_df(operation, date.month, date.year, responsible))
    
    return pd.concat(dfs)


In [23]:
ap = download_file(url)

In [24]:
with pdfplumber.open(ap) as pdf:
        pages = pdf.pages
        text = ""
        for page in pages:
            text += page.extract_text()

In [25]:
print(text)

FORMULÁRIO CONSOLIDADO
Negociação de Administradores e Pessoas Ligadas - Art. 11 – Instrução CVM nº 358/2002
Em 11/2021
( ) ocorreram somente as seguintes operações com valores mobiliários e derivativos, de acordo com o Art. 11 da Instrução CVM
nº 358/2002.
(X) não foram realizadas operações com valores mobiliários e derivativos, de acordo com o Art. 11 da Instrução CVM nº
358/2002, sendo que possuo as seguintes posições dos valores mobiliários e derivativos.
Denominação da Companhia: VALE S.A.
( )
Grupo e Pessoas
( X ) Controlador ( ) Conselho Administração Direto ( ) Conselho Fiscal ( ) Órgãos Técnicos ou Consultivos
Ligadas
ria
Saldo Inicial
Valor Mobiliário Derivativo Características dos Títulos Quantidade
Ações ON 0
Movimentações no Mês
Valor Características dos
Intermediário Operação Dia Quantidade Preço Volume (R$)
Mobiliário/Derivativo Títulos
Saldo Final
Valor Mobiliário Derivativo Características dos Títulos Quantidade
Ações ON 0FORMULÁRIO CONSOLIDADO
Negociação de Administra

In [45]:
import re

def extract_parts(text):
    # Extract Saldo Inicial
    saldo_inicial = re.findall(r'Saldo Inicial(.*?)Movimentações no Mês', text, re.DOTALL)

    # Extract Saldo Final
    saldo_final = re.findall(r'Saldo Final(.*?)(FORMULÁRIO CONSOLIDADO|$)', text, re.DOTALL)
    saldo_final = [t[0] for t in saldo_final]
    
    # Extract Member
    members = re.findall(r'Grupo e Pessoas\s*(.*?)Ligadas', text, re.DOTALL)

    member = []
    for m in members:
        match = re.search(r'\( X \)(.*?)(\( |$)', m, re.DOTALL)
        if match is not None:
            member.append(match.group(1).strip())
        else:
            member.append(None)
    
    # Special case for 'Diretoria'
    if "( X )\nGrupo e Pessoas" in text:
        member = ["Diretoria" if m is None else m for m in member]

    return saldo_inicial, saldo_final, member

saldo_inicial, saldo_final, member = extract_parts(text)



In [46]:
print(member)

['Controlador', 'Conselho Administração Direto', 'Diretoria', 'Conselho Fiscal', 'Órgãos Técnicos ou Consultivos']


In [11]:
def extract_parts(text):
    # Extract Saldo Inicial
    saldo_inicial = re.findall(r'Saldo Inicial(.*?)Movimentações no Mês', text, re.DOTALL)

    # Extract Saldo Final
    saldo_final = re.findall(r'Saldo Final(.*?)(FORMULÁRIO CONSOLIDADO|$)', text, re.DOTALL)

    saldo_final = [t[0] for t in saldo_final]


    return saldo_inicial, saldo_final


def extract_info(text):
    # Extract the class and quantity
    matches = re.findall(r'((?:\w+ ?){1,4}) (\d{1,3}(?:\.\d{3})*)', text)

    # Convert the matches to a list of lists, removing the dots from the quantities
    info = [[match[0].strip(), int(match[1].replace('.', ''))] for match in matches]

    return info

def create_df(saldo_inicial_list, saldo_final_list, members, date):
    df_final = pd.DataFrame(columns=["Class", "Initial Qty", "Final Qty", "Net Qty", "Member", "Date"])   

    for saldo_inicial, saldo_final, member in zip(saldo_final_list, saldo_inicial_list, members): 
        saldo_inicial = pd.DataFrame(saldo_inicial, columns=["Class", "Initial Qty"])
        saldo_final = pd.DataFrame(saldo_final, columns=["Class", "Final Qty"])

        df = pd.merge(saldo_inicial, saldo_final, on="Class")
        df["Net Qty"] = df["Final Qty"] - df["Initial Qty"]
        df["Member"] = member
        df["Date"] = date

        df_final = pd.concat([df_final, df])
    
    return df_final




In [13]:
extract_info(text)

[['Instrução CVM nº', 358],
 ['Em', 2],
 ['nº', 358],
 ['Ações ON', 155085814],
 ['Outros ADR', 1106499],
 ['Ações ON', 155085814],
 ['Outros ADR', 1106499]]

In [14]:
print(saldo_final[0])


Valor Mobiliário Derivativo Características dos Títulos Quantidade
Ações ON 155.085.814
Outros ADR 1.106.499


In [15]:
extract_info(saldo_final[0])

[['Ações ON', 155085814], ['Outros ADR', 1106499]]

In [16]:
saldo_inicial_text_list, saldo_final_text_list = extract_parts(text)

saldo_inicial_list = []
saldo_final_list = []

for saldo_final_text, saldo_inicial_text in zip(saldo_final_text_list, saldo_inicial_text_list):
    saldo_inicial_list.append(extract_info(saldo_final_text))
    saldo_final_list.append(extract_info(saldo_inicial_text))


members = ["Conselho Administração", "Diretoria", "Conselho Fiscal"]

date = extract_datetime(text)

df = create_df(saldo_inicial_list, saldo_final_list, members, date) 

df

,Class,Initial Qty,Final Qty,Net Qty,Member,Date
0,Ações ON,155085814,155085814,0,Conselho Administração,2020-02-01
1,Outros ADR,1106499,1106499,0,Conselho Administração,2020-02-01


In [17]:
operations_list = extract_operations(text, "Movimentações no Mês", "Saldo Final")

generate_trade_df(operations_list, extract_datetime(text))

,Valor Mobiliário/Derivativo,Características dos Títulos,Intermediário,Operação,Dia,Quantidade,Preço,Volume (R$),Mês,Ano,Responsável


In [18]:
print(operations_list[0])

Movimentações no Mês
Valor Características dos
Intermediário Operação Dia Quantidade Preço Volume (R$)
Mobiliário/Derivativo Títulos
Saldo Final


In [19]:
generate_operations_df(operations_list[0])

TypeError: generate_operations_df() missing 3 required positional arguments: 'month', 'year', and 'responsible'

,Valor Mobiliário/Derivativo,Características dos Títulos,Intermediário,Operação,Dia,Quantidade,Preço,Volume (R$)
0,Outros,ADR ORDINARIA,JP Morgan,Venda à vista,4,100.000,"13,95775","1.395.775,20"
1,Outros,ADR ORDINARIA,JP Morgan,Venda à vista,9,100.000,"13,77503","1.377.502,50"
2,Outros,ADR ORDINARIA,JP Morgan,Venda à vista,21,100.000,"14,15910","1.415.910,00"
3,Outros,ADR ORDINARIA,JP Morgan,Venda à vista,21,100.000,"14,16608","1.416.607,70"
4,Outros,ADR ORDINARIA,JP Morgan,Venda à vista,22,200.000,"14,26967","2.853.934,20"
5,Outros,ADR ORDINARIA,JP Morgan,Venda à vista,28,315.000,"14,37060","4.526.739,00"
6,Ações,ON,Direto c/ a Cia,Exercício de Opções,30,238.990,"15,20000","3.632.648,00"


In [82]:
import re

text = '''
Entrega Ações
Ações ON Direto c/ a Cia Restritas 16 27.341 14,68000 401.365,88
(PSU)
'''

pattern = re.compile(r"(Ações)\s+(ON)\s+(.+?)\s+(.+?)\s+(\d+)\s+([\d\.,]+)\s+([\d\.,]+)\s+([\d\.,]+)\n\((PSU)\)")
matches = pattern.findall(text)

for match in matches:
    operation = [match[0], match[1], match[2], 'Entrega Ações '+match[3]+' ('+match[8]+')', match[4], match[5], match[6], match[7]]
    print(tuple(operation))


('Ações', 'ON', 'Direto', 'Entrega Ações c/ a Cia Restritas (PSU)', '16', '27.341', '14,68000', '401.365,88')


In [114]:
dd = pd.read_csv("ipe_cia_aberta_2010.csv", encoding="latin1", sep=";")

dd.head()

,CNPJ_Companhia,Nome_Companhia,Codigo_CVM,Data_Referencia,Categoria,Tipo,Especie,Assunto,Data_Entrega,Tipo_Apresentacao,Protocolo_Entrega,Versao,Link_Download
0,00.000.000/0001-91,BANCO DO BRASIL S.A.,1023,2009-03-31,Assembleia,AGE,Ata,"Aumento do Capital Social (inciso I, art. 166 ...",2010-01-04,RE - Reapresentação Espontânea,NaN,NaN,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
1,00.000.000/0001-91,BANCO DO BRASIL S.A.,1023,2009-04-23,Assembleia,AGE,Ata,Alteração do Estatuto Social do BB||Aprovação ...,2010-01-04,RE - Reapresentação Espontânea,NaN,NaN,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
2,00.000.000/0001-91,BANCO DO BRASIL S.A.,1023,2009-08-18,Assembleia,AGE,Ata,"Aumento do Capital Social (inciso I, art. 166 ...",2010-01-04,RE - Reapresentação Espontânea,NaN,NaN,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
3,00.000.000/0001-91,BANCO DO BRASIL S.A.,1023,2009-11-30,Assembleia,AGE,Ata,Alteração Estatuto BB||Aprovação Protocolo e J...,2010-05-10,RE - Reapresentação Espontânea,NaN,NaN,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
4,00.000.000/0001-91,BANCO DO BRASIL S.A.,1023,2007-10-23,Assembleia,AGE,Ata,Antecipação do Direito de Subscrição do Bônus ...,2010-05-17,RE - Reapresentação Espontânea,NaN,NaN,https://www.rad.cvm.gov.br/ENET/frmDownloadDoc...
